# Transformer related code practice

In [62]:
import numpy as np

## Softmax Calculation
### Formula
$$Softmax(i) = \frac{e^i}{\sum_{j=1}^{n}e^j}$$

### Function
#### `np.sum(x, axis=axis, keepdims=True)中的axis参数决定了沿哪个维度进行求和：`

axis=0：按列求和，压缩第 0 维（行）。

axis=1：按行求和，压缩第 1 维（列）。

axis=-1：按最后一维求和（适用于多维数组）。

keepdims=True：保持结果的维度与输入一致，仅将求和的维度大小变为 1。



In [76]:

def softmax(x , axis):
	"""
	Compute softmax of array x.
	"""
	exp_x = np.exp(x-np.max(x,axis=axis, keepdims=True))
	return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

a = [i for i in range(0,101)]
print(a)

softmax(a, 0)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]


array([2.35153650e-44, 6.39213895e-44, 1.73756352e-43, 4.72318733e-43,
       1.28389543e-42, 3.48998961e-42, 9.48677535e-42, 2.57877290e-41,
       7.00983153e-41, 1.90546977e-40, 5.17960384e-40, 1.40796230e-39,
       3.82723833e-39, 1.04035124e-38, 2.82796788e-38, 7.68721369e-38,
       2.08960133e-37, 5.68012532e-37, 1.54401814e-36, 4.19707646e-36,
       1.14088367e-35, 3.10124334e-35, 8.43005342e-35, 2.29152610e-34,
       6.22901377e-34, 1.69322149e-33, 4.60265322e-33, 1.25113086e-32,
       3.40092628e-32, 9.24467611e-32, 2.51296351e-31, 6.83094304e-31,
       1.85684283e-30, 5.04742213e-30, 1.37203159e-29, 3.72956853e-29,
       1.01380184e-28, 2.75579911e-28, 7.49103864e-28, 2.03627542e-27,
       5.53517048e-27, 1.50461533e-26, 4.08996852e-26, 1.11176871e-25,
       3.02210068e-25, 8.21492137e-25, 2.23304715e-24, 6.07005148e-24,
       1.65001106e-23, 4.48519509e-23, 1.21920243e-22, 3.31413582e-22,
       9.00875516e-22, 2.44883355e-21, 6.65661973e-21, 1.80945684e-20,
      

In [67]:

def compute_qkv(X, W_q, W_k, W_v):
	"""
	Compute Q, K, V matrices from input X using the provided weight matrices.
	"""
	Q = np.dot(X, W_q)
	K = np.dot(X, W_k)
	V = np.dot(X, W_v)

	return Q, K, V
X = np.array([[1, 0], [0, 1]])
W_q = np.array([[1, 0], [0, 1]])
W_k = np.array([[1, 0], [0, 1]])
W_v = np.array([[1, 2], [3, 4]])

compute_qkv(X, W_q, W_k, W_v)




(array([[1, 0],
        [0, 1]]),
 array([[1, 0],
        [0, 1]]),
 array([[1, 2],
        [3, 4]]))

### Self Attention

$$Attention(Q, K, V) = Softmax(\frac{Q K^T}{\sqrt{d_K}})$$

$ Q=XW_q $  &emsp; &emsp; $ K=XW_k $ &emsp; &emsp; $ V=XW_v $
在深度学习中，高维矩阵乘法（如批量矩阵乘法）是常见操作，但维度变化容易混淆。以下是详细解释和示例：


### **1. 二维矩阵乘法（基础）**
对于二维矩阵 $A$ 和 $B$：
- $A$ 的形状为 `(m, n)`
- $B$ 的形状为 `(n, p)`
- 结果 $C = A \times B$ 的形状为 `(m, p)`

**示例**：
```python
A = np.random.randn(3, 4)  # 形状: (3, 4)
B = np.random.randn(4, 5)  # 形状: (4, 5)
C = np.dot(A, B)          # 形状: (3, 5)
```


### **2. 高维矩阵乘法（批量矩阵）**
在深度学习中，通常需要处理批量数据，例如：
- 输入序列形状为 `(batch_size, seq_len, hidden_dim)`

此时的矩阵乘法称为**批量矩阵乘法**，需满足：
- 最后两个维度满足二维矩阵乘法规则
- 前面的维度必须完全相同或为1（广播机制）

#### **规则**：
假设两个高维矩阵 $A$ 和 $B$：
- $A$ 的形状为 `(a₁, a₂, ..., aₖ, m, n)`
- $B$ 的形状为 `(a₁, a₂, ..., aₖ, n, p)`
- 结果 $C = A \times B$ 的形状为 `(a₁, a₂, ..., aₖ, m, p)`


### **3. NumPy中的批量矩阵乘法**
在NumPy中，可以使用以下方式进行批量矩阵乘法：
1. **`np.matmul` 或 `@` 运算符**：专门处理批量矩阵乘法
2. **`np.einsum`**：更灵活的爱因斯坦求和表示法


### **4. 常见场景示例**
#### **场景1：自注意力机制中的QKV计算**
- $Q$ 的形状：`(batch_size, seq_len, hidden_dim)`
- $K$ 的形状：`(batch_size, seq_len, hidden_dim)`
- 计算 $QK^T$：
  ```python
  attention_scores = np.matmul(Q, K.transpose(0, 2, 1))
  # Q: (batch_size, seq_len, hidden_dim)
  # K.transpose(0, 2, 1): (batch_size, hidden_dim, seq_len)
  # 结果: (batch_size, seq_len, seq_len)
  ```

#### **场景2：多头注意力中的每个头**
假设 `num_heads = 8`，`hidden_dim = 512`，则每个头的维度为 `512/8 = 64`：
- $Q$ 的形状：`(batch_size, seq_len, num_heads, head_dim)`
- $K$ 的形状：`(batch_size, seq_len, num_heads, head_dim)`
- 计算 $QK^T$：
  ```python
  # 调整维度：将num_heads移到batch_size维度前
  Q_reshaped = Q.transpose(0, 2, 1, 3).reshape(batch_size*num_heads, seq_len, head_dim)
  K_reshaped = K.transpose(0, 2, 1, 3).reshape(batch_size*num_heads, seq_len, head_dim)
  
  # 批量矩阵乘法
  attention_scores = np.matmul(Q_reshaped, K_reshaped.transpose(0, 2, 1))
  # 结果形状: (batch_size*num_heads, seq_len, seq_len)
  ```


### **5. 使用 `einsum` 简化高维运算**
`einsum` 可以更清晰地表达复杂的矩阵运算：
```python
# 自注意力中的QK^T计算
attention_scores = np.einsum('bih,bjh->bij', Q, K)
# 含义：
# b: batch_size
# i, j: 序列位置
# h: hidden_dim
# 结果形状: (batch_size, seq_len, seq_len)

# 多头注意力中的QK^T计算
attention_scores = np.einsum('bhid,bhjd->bhij', Q, K)
# 含义：
# b: batch_size
# h: num_heads
# i, j: 序列位置
# d: head_dim
# 结果形状: (batch_size, num_heads, seq_len, seq_len)
```


### **6. 广播机制在矩阵乘法中的应用**
如果两个矩阵的某些维度不匹配，但其中一个为1，则会触发广播机制：
```python
A = np.random.randn(2, 3, 4, 5)  # 形状: (2, 3, 4, 5)
B = np.random.randn(2, 1, 5, 6)  # 形状: (2, 1, 5, 6)
C = np.matmul(A, B)              # 形状: (2, 3, 4, 6)
# B的第二个维度为1，会自动广播为3以匹配A
```


### **总结**
1. **二维矩阵乘法**：`(m, n) × (n, p) → (m, p)`
2. **批量矩阵乘法**：最后两维满足二维规则，前面的维度必须相同或为1
3. **常用方法**：
   - `np.matmul` 或 `@`：处理批量矩阵乘法
   - `np.einsum`：灵活表达复杂运算
4. **注意转置操作**：如自注意力中的 `K.transpose(0, 2, 1)`
5. **广播机制**：当维度为1时自动扩展匹配

理解这些规则后，高维矩阵乘法的维度变化将变得清晰可控。

- $ W_q \in \mathbb{R}^{d \times d_k}$：查询权重矩阵

- $ W_k \in \mathbb{R}^{d \times d_k}$：键权重矩阵

- $ W_v \in \mathbb{R}^{d \times d_v}$：值权重矩阵

- $ d_k$ 和 $d_v$ 分别是键和值的维度，通常 $d_k = d_v = d/h$（h 是注意力头数）
​



In [ ]:
def self_attention(Q, K, V):
	"""
	在没有batch-size维度的情况下, 计算attention可以直接点乘
	"""
	attention_scores = np.dot(Q, K.T)/np.sqrt(Q.shape[-1])
	attention_weights = softmax(attention_scores, 1)
	attention_output = np.dot(attention_weights, V)

def self_attention_with_batch(Q, K, V):
	"""
	在有batch-size维度的情况下, 计算attention需要将batch-size维度展开
	"""
	#batch_size维度保持不变，seq_len, d_model维度转置
	attention_scores = np.matmul(Q, K.transpose(0, 2, 1))/np.sqrt(Q.shape[-1])
	attention_weights = softmax(attention_scores, 1)
	attention_output = np.dot(attention_weights, V)



	return attention_output

Q , K , V = compute_qkv(X, W_q, W_k, W_v)

self_attention(Q, K, V)

array([[1.53788284, 2.53788284],
       [2.46211716, 3.46211716]])

In [84]:
# Sigmoid Activation Function
import numpy as np

def sigmoid(z: float) -> float:
	#Your code here
    result = round(1/(1 + np.e**(-z)),4)
    return result

print(sigmoid(0.2))

0.5498


这道题目要求实现一个基于梯度下降的线性回归函数，核心目标是通过迭代优化找到最佳的回归系数（包括截距项），使模型对目标值的预测尽可能准确。下面详细解析题意和解决方法：


### **一、题意解析**
#### **1. 输入与输出**
- **输入**：
  - `X`：特征矩阵（NumPy数组），其中已包含一列全为1的列（用于表示线性回归中的截距项，即常数项）。假设`X`的形状为`(m, n)`，其中`m`是样本数量，`n`是特征数量（含截距项对应的列）。
  - `y`：目标值数组（NumPy数组），形状为`(m,)`，表示每个样本对应的真实值。
  - `alpha`：学习率（步长），控制每次参数更新的幅度。
  - `iterations`：迭代次数，即梯度下降的总步数。
- **输出**：
  - 线性回归模型的系数（NumPy数组），形状为`(n,)`，包含截距项和各特征的系数，结果需保留4位小数。

#### **2. 核心任务**
通过梯度下降算法最小化线性回归的损失函数（均方误差），求解最优参数`θ`（系数向量），使得模型预测值`y_pred = X·θ`尽可能接近真实值`y`。


### **二、线性回归与梯度下降的核心原理**
#### **1. 线性回归模型**
线性回归的数学表达式为：  
$$\hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + ... + \theta_{n-1} x_{n-1}$$  
其中：
- $\hat{y}$ 是预测值，$\theta_0$ 是截距项（常数项），$\theta_1, ..., \theta_{n-1}$ 是各特征的系数。
- 若将特征矩阵`X`扩展为包含一列全为1的列（用于表示$\theta_0$的系数），则模型可简化为矩阵形式：  
  $$\hat{y} = X \cdot \theta$$  
  其中`X`的形状为`(m, n)`，`θ`的形状为`(n,)`，$\hat{y}$的形状为`(m,)`。

#### **2. 损失函数**
线性回归使用**均方误差（MSE）** 作为损失函数，衡量预测值与真实值的平均偏差：  
$$J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} (\hat{y}_i - y_i)^2$$  
（公式中除以`2m`而非`m`是为了求导后简化计算，不影响最优解）。

#### **3. 梯度下降算法**
梯度下降是一种迭代优化方法，通过不断沿着损失函数的负梯度方向更新参数，逐步减小损失值。核心步骤如下：  
1. **初始化参数**：通常将`θ`初始化为全零向量。  
2. **计算梯度**：损失函数对`θ`的梯度（偏导数）为：  
   $$\nabla J(\theta) = \frac{1}{m} X^T \cdot (X \cdot \theta - y)$$  
   其中`X·θ - y`是预测值与真实值的误差向量。  
3. **更新参数**：沿着负梯度方向调整`θ`：  
   $$\theta = \theta - \alpha \cdot \nabla J(\theta)$$  
   （`α`为学习率，控制更新幅度）。  
4. **迭代优化**：重复步骤2和3，直到达到指定迭代次数，最终得到的`θ`即为最优系数。


### **三、解决方法（步骤拆解）**
#### **1. 初始化参数**
- 系数向量`θ`的长度等于特征矩阵`X`的列数（`n`），初始化为全零向量（`np.zeros(n)`）。  
  （注：初始化为零不影响最终结果，因为线性回归的损失函数是凸函数，只有一个全局最优解。）

#### **2. 迭代更新参数**
对于每一次迭代（共`iterations`次）：  
- **步骤1：计算预测误差**  
  预测值`y_pred = X·θ`（用矩阵乘法`np.dot(X, θ)`或`X @ θ`实现），误差向量为`error = y_pred - y`。  
- **步骤2：计算梯度**  
  根据梯度公式，梯度`gradient = (X^T · error) / m`（`X^T`是`X`的转置，`m`是样本数）。  
- **步骤3：更新系数**  
  按照`θ = θ - α * gradient`更新参数，沿着负梯度方向减小损失。












In [93]:
import numpy as np

def linear_regression_gradient_descent(X: np.ndarray, y: np.ndarray, alpha: float, iterations: int) -> np.ndarray:
	m, n = X.shape
	theta = np.zeros((n, 1))
	for _ in range(iterations):
		error = np.dot(X, theta) - y
		gradient = np.dot(X.T, error) / m
		theta = theta - alpha * gradient
	return np.round(theta, 4)

X = np.array([[1, 1], [1, 2], [1, 3]])
y = np.array([[1], [2], [3]])
alpha = 0.01
iterations = 1000

linear_regression_gradient_descent(X, y, alpha, iterations)

array([[0.1107],
       [0.9513]])